In [0]:
CREATE or REFRESH STREAMING TABLE raw_transactions_streaming (
    CONSTRAINT correct_schema EXPECT (_rescued_data IS NULL)
)
COMMENT 'This streaming table has all data from json files'
AS
SELECT *
FROM STREAM cloud_files(
    'dbfs:/tmp/streaming_landing_store',
    'json',
    map('cloudFiles.inferColumnTypes', 'true')
);

In [0]:
CREATE OR REFRESH STREAMING table dim_customer (
  customer_id STRING NOT NULL,
  customer_name STRING,
  customer_email STRING,
  customer_phone STRING,
  customer_birthdate STRING,
  customer_gender STRING,
  CONSTRAINT valid_customer_id EXPECT (customer_id IS NOT NULL)
)
AS
SELECT
  customer_id,
  customer_name,
  customer_email,
  customer_phone,
  customer_birthdate,
  customer_gender
FROM stream(databricksws.default.raw_transactions_streaming)
GROUP BY customer_id, customer_name, customer_email, customer_phone, customer_birthdate, customer_gender;

In [0]:
CREATE OR REFRESH STREAMING TABLE dim_product (
  product_id STRING NOT NULL,
  product_name STRING,
  product_category STRING,
  product_price DOUBLE,
  CONSTRAINT valid_product_id EXPECT (product_id IS NOT NULL)
)
AS
SELECT
  product_id,
  product_name,
  product_category,
  cast(product_price AS double) as product_price
FROM STREAM(databricksws.default.raw_transactions_streaming)
GROUP BY product_id, product_name, product_category, product_price;

In [0]:
CREATE OR REFRESH STREAMING TABLE dim_store (
  store_id STRING NOT NULL,
  store_name STRING,
  store_city STRING,
  store_state STRING,
  store_country STRING,
  store_postcode STRING,
  CONSTRAINT valid_store_id EXPECT (store_id IS NOT NULL)
)
AS
SELECT
  store_id,
  store_name,
  store_city,
  store_state,
  store_country,
  store_postcode
FROM STREAM(databricksws.default.raw_transactions_streaming)
GROUP BY store_id, store_name, store_city, store_state, store_country, store_postcode;

In [0]:
CREATE OR REFRESH STREAMING TABLE fact_transaction (
  transaction_id STRING NOT NULL,
  customer_id STRING,
  store_id STRING,
  product_id STRING,
  transaction_timestamp TIMESTAMP,
  transaction_date STRING,
  transaction_hour INT,
  transaction_day_of_week STRING,
  transaction_month STRING,
  transaction_year INT,
  quantity INT,
  total_amount DOUBLE,
  payment_method STRING,
  CONSTRAINT valid_transaction_id EXPECT (transaction_id IS NOT NULL)
)
AS
SELECT
  CAST(transaction_id AS STRING) AS transaction_id,
  CAST(customer_id AS STRING) AS customer_id,
  CAST(store_id AS STRING) AS store_id,
  CAST(product_id AS STRING) AS product_id,
  TRY_CAST(transaction_timestamp AS TIMESTAMP) AS transaction_timestamp,
  CAST(transaction_date AS STRING) AS transaction_date,
  TRY_CAST(transaction_hour AS INT) AS transaction_hour,
  CAST(transaction_day_of_week AS STRING) AS transaction_day_of_week,
  CAST(transaction_month AS STRING) AS transaction_month,
  TRY_CAST(transaction_year AS INT) AS transaction_year,
  TRY_CAST(quantity AS INT) AS quantity,
  TRY_CAST(total_amount AS DOUBLE) AS total_amount,
  CAST(payment_method AS STRING) AS payment_method
FROM STREAM(databricksws.default.raw_transactions_streaming);